In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!cp /kaggle/input/quora-question-pairs/train.csv.zip .
!unzip train.csv.zip
!ls

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import nltk
from matplotlib import rcParams
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.cm as cm
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import collections
%matplotlib inline

In [ ]:
df = pd.read_csv("train.csv")

print("Number of data points:",df.shape[0])

# Loading Data And Getting Basics Stats

In [ ]:
df.head(5)

In [ ]:
df.info()

### Here we can see that we have some missing value in question1 (1 Question) and question2 (2 Question)

<h3> 3.2.1 Distribution of data points among output classes</h3>

In [ ]:
# Checking if we have imbalance data or not
df.groupby("is_duplicate")['id'].count().plot.bar()

### Checking if dataset is balanced or imbalanced

In [ ]:
unique=len(set(list(df['qid1'])+list(df['qid2'])))
print ('Total number of  Unique Questions are: {}\n'.format(unique))

### Percentage of Similar question and Non-Similar Question

In [ ]:
print('-->> Question pairs are not Similar (is_duplicate = 0):\n     {}%'.format(100 - round(df['is_duplicate'].mean()*100, 2)))
print('-->> Question pairs are Similar (is_duplicate = 1):\n     {}%'.format(round(df['is_duplicate'].mean()*100, 2)))

In [ ]:
qids = pd.Series(list(df['qid1']) + list(df['qid2'])) #Creating a dataframe that contains the question id of both quid1 and quid2
unique_qs = len(np.unique(qids))  #Numpy array to filter down all qniue elements
qs_morethan_onetime = np.sum(qids.value_counts() > 1)   # Counts any question that have been repeated more than one time
print ('Total number of  Unique Questions are: {}\n'.format(unique_qs))
#len(set(list(df['qid1'])+list(df['qid2'])))


print ('Number of unique questions that appear more than one time: {} ({}%)\n'.format(qs_morethan_onetime,round(qs_morethan_onetime/unique_qs*100,2)))

print ('Max number of times a single question is repeated: {}\n'.format(max(qids.value_counts()))) # Taking the frequency of all question and printing the max of them

q_vals=qids.value_counts()

q_vals=q_vals.values

In [ ]:
x = ["unique_questions" , "Repeated Questions"]
y =  [unique_qs , qs_morethan_onetime]

plt.figure(figsize=(8, 6))
plt.title ("Plot representing unique and repeated questions  ")
sns.barplot(x,y)
plt.show()

In [ ]:
#checking whether there are any repeated pair of questions

duplicateRowsDF = df[df.duplicated(['qid1','qid2'])]   # Collecting all Duplicate data into a dataframe so than we can also see what duplicate value we have

print ("Number of duplicate questions : ",duplicateRowsDF.shape[0])

## Plotting Questions based on there frequency

In [ ]:
plt.figure(figsize=(30, 10))

plt.hist(qids.value_counts(), bins=250)

plt.yscale('log')

plt.title('Log-Histogram of question appearance counts')

plt.xlabel('Number of occurences of question')

plt.ylabel('Number of questions')

print ('Maximum number of times a single question is repeated: {}\n'.format(max(qids.value_counts()))) 

## Checking and Removing Null Value

In [ ]:
df[df.isnull().any(1)] # Checking if any value is null in our dataset


### As we have very few points with NULL value then its better to remove them

In [ ]:
df.dropna(inplace=True)  # Droping Null Value
df[df.isnull().any(1)] 


#  Basic Feature Extraction

In [ ]:
df['Q1_Len']=df['question1'].str.len()  # Finding length of Question 1
df['Q2_Len']=df['question2'].str.len()  # Finding length of Question 2

df['Q1_Words']=df['question1'].apply(lambda row : len(row.split()))   # Finding Number of Words in Question 1
df['Q2_Words']=df['question2'].apply(lambda row : len(row.split()))   # Finding Number of Words in Question 2

# Fucntion to find the number of common words in Question 1 and Question 2

def common(row):
    '''
    We are converting both Question 1 and Question 2 to set (and also converting them to lower so that every word have same) and finding there intersection so that we can get common words
    Then we are simply finding the lenth of those common words
    '''
    return len((set(row['question1'].lower().split())).intersection(set(row['question2'].lower().split())))


# Creating New Column with Number of Common Words
df['common_Word'] = df.apply(common, axis=1)


# Total Number of Distict words in both question1 and question2

def total(row):
    '''
    Coverting them to lower form then removing extra spaces and them removing the repeted words by converting them to sets 
    Then finding Length of Both the questions and adding them to find total words in both
    '''
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

# Getting total number of (unique) words in both question1 and question2
df['word_Total'] = df.apply(total, axis=1)


def word_share(row):
    '''
    Here we are finding total number of shared word and dividing by total number of words [ (A intersection B)/A+B]
    '''
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))    # Finding the number of common words between question1 and question2 and dividing by total words between both of them
df['Shared_Word'] = df.apply(word_share, axis=1)


# Saving our dataframe as csv file
df.to_csv("With_Feature.csv", index=False)
df.head()

# Analysing our extracted features

## Asking Some Basic Question To Our Extracted Feature

In [ ]:
print ("Minimum length of the questions in question1 : " , min(df['Q1_Words']))
print ("Minimum length of the questions in question2 : " , min(df['Q2_Words']))

print ("Number of Questions with minimum length [question1] :", df[df['Q1_Words']== 1].shape[0])
print ("Number of Questions with minimum length [question2] :", df[df['Q2_Words']== 1].shape[0])

print ("Maximum length of the questions in question1 : " , max(df['Q1_Words']))
print ("Maximum length of the questions in question2 : " , max(df['Q2_Words']))

print ("Number of Questions with minimum length [question1] :", df[df['Q1_Words']>120].shape[0])
print ("Number of Questions with minimum length [question2] :", df[df['Q2_Words']> 230].shape[0])

print ("Maximum number of Common word : " , max(df['common_Word']))
print ("Maximum number of Shared Word : " , max(df['Shared_Word']))


## Analysing Shared Word

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(1,2,1)
sns.violinplot(x = 'is_duplicate', y = 'Shared_Word', data = df[0:])

plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate'] == 1.0]['Shared_Word'][0:] , label = "1", color = 'red')
sns.distplot(df[df['is_duplicate'] == 0.0]['Shared_Word'][0:] , label = "0" , color = 'green' )
plt.show()


## [2]Text pre-processing


In [ ]:
from nltk.stem import SnowballStemmer
snow=nltk.stem.SnowballStemmer('english')

stopwords= set(['the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


In [ ]:
def removeStopWord(word):
  token=word.split(" ")   ## coverting string to token (list of word) \\ like ["this","is","token"]
  removestop=[snow.stem(x) for x in token if x not in stopwords]   ##removing stopwords and also doing Stemming
  removed=" ".join(removestop)  ##joing back the list into sentence
  return removed

In [ ]:

def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " is")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("@","at")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    #Remove any special character like [@ , ' ; "" ']
    
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    #Removing Stopwords And Doing Stemming
    x=removeStopWord(x)
    
               
    
    return x